# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

OpenAI API Key exists and begins sk-proj-


In [3]:
# Connect to client libraries

openai = OpenAI()


In [17]:
OPENAI_MODEL = "gpt-5"

# Want to keep costs ultra-low? Uncomment these lines:

# OPENAI_MODEL = "gpt-5-nano"


In [15]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Windows',
  'arch': 'AMD64',
  'release': '11',
  'version': '10.0.26100',
  'kernel': '11',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'x86_64-w64-mingw32'},
 'package_managers': ['winget'],
 'cpu': {'brand': 'AMD Ryzen 9 6900HS with Radeon Graphics',
  'cores_logical': 16,
  'cores_physical': 8,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'gcc.EXE (Rev3, Built by MSYS2 project) 13.2.0',
   'g++': 'g++.EXE (Rev3, Built by MSYS2 project) 13.2.0',
   'clang': '',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': ''},
  'linkers': {'ld_lld': ''}}}

In [ ]:
message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}

compile_command = [ "g++", "-std=c++20", "-O3", "-march=native", "-mtune=native", "-flto", "-DNDEBUG", "-pipe", "-pthread", "main.cpp", "-o", "main.exe" ]

run_command = ["main.exe"]

i got this system information from running the same code as above: run again to be sure.
"""

response = openai.chat.completions.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))
    

You’re already set. Your system has g++ 13.2.0 (MSYS2/MinGW-w64), so you don’t need to install a C++ compiler to build and run a single main.cpp.

Use these exact commands in Python:

compile_command = ["g++", "-std=c++20", "-O3", "-march=native", "-mtune=native", "-flto", "-DNDEBUG", "-pipe", "main.cpp", "-o", "main.exe"]
run_command = ["main.exe"]

Notes:
- The above aims for fast runtime (-O3, -march/-mtune=native, -flto). -pipe only speeds compilation.
- Only add "-pthread" if your code actually uses threads; on Windows it adds a dependency on libwinpthread-1.dll.
- If Python can’t find g++, ensure C:\msys64\mingw64\bin (or your MSYS2 mingw64 bin) is on PATH. If needed at runtime, that path should also be on PATH so dependent DLLs are found.

Optional: If you ever need to install the toolchain
- winget install MSYS2.MSYS2
- Open the “MSYS2 MinGW x64” (or “MSYS2 UCRT64”) shell and run: pacman -S --needed base-devel mingw-w64-ucrt-x86_64-gcc
- Add C:\msys64\ucrt64\bin (or mingw64\bin) to your Windows PATH.
- Verify with: g++ --version

## If you need to install something

If you would like to, please follow GPTs instructions! Then rerun the analysis afterwards (you might need to Restart the notebook) to confirm you're set.

You should now be equipped with the command to compile the code, and the command to run it!

Enter that in the cell below:

In [6]:
# compile_command = ["clang++", "-std=c++17", "-Ofast", "-mcpu=native", "-flto=thin", "-fvisibility=hidden", "-DNDEBUG", "main.cpp", "-o", "main"]
# run_command = ["./main"]

compile_command = ["g++", "-std=c++20", "-O3", "-march=native", "-mtune=native", "-flto", "-DNDEBUG", "-pipe", "main.cpp", "-o", "main.exe"] 
run_command = ["main.exe"]

## And now, on with the main task

In [7]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [8]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [9]:
def write_output(cpp):
    with open("main.exe", "w", encoding="utf-8") as f:
        f.write(cpp)

In [ ]:
def port(python):
    reasoning_effort = "high" if 'gpt' in OPENAI_MODEL else None
    response = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)


In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [14]:
def run_python(code):
    globals = {"__builtins__": __builtins__}
    exec(code, globals)

In [15]:
run_python(pi)

Result: 3.141592656089
Execution Time: 21.353750 seconds


In [ ]:
port(pi)


# Compiling C++ and executing

This next cell contains the command to compile a C++ file based on the instructions from GPT.
 
> Not an exact comparison but you can still get the idea of performance difference.  
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [24]:
# Use the commands from GPT 5

def compile_and_run():
    subprocess.run(compile_command, check=True, text=True, capture_output=True)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [ ]:
compile_and_run()

In [ ]:
19.178207/0.082168

In [ ]:
print(f"""
In this experiments, the performance speedups were:

GPT-5: {19.178207/0.082168:.0f}X speedup

""")

In [13]:
import gradio as gr

In [18]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=28, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=28)
    with gr.Row():
        convert = gr.Button("Convert code")

    convert.click(port, inputs=[python], outputs=[cpp])

ui.launch(inbrowser=True)


c:\Users\srina\projects\llm_projects\python_to_optimized_C++_LLM\.venv\Lib\site-packages\gradio\utils.py:1052: UserWarning: Expected 3 arguments for function <function port at 0x0000028D337F9800>, received 1.
  warnings.warn(
c:\Users\srina\projects\llm_projects\python_to_optimized_C++_LLM\.venv\Lib\site-packages\gradio\utils.py:1056: UserWarning: Expected at least 3 arguments for function <function port at 0x0000028D337F9800>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
